### Scenario 1:
Total size of serialized results of 16 tasks (1048.5 MB) is bigger than spark.driver.maxResultSize (1024.0 MB)

`py4j.protocol.Py4JJavaError: An error occurred while calling o171.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 16 tasks (1048.5 MB) is bigger than spark.driver.maxResultSize (1024.0 MB)`

[Link: stack overflow](https://stackoverflow.com/questions/47996396/total-size-of-serialized-results-of-16-tasks-1048-5-mb-is-bigger-than-spark-dr)

**Analysis:**
1. You called collect method
2. Spark keep sending data back to drive --> accumulators, data for broadcast joins and when scanning lots of (20k+) partitions to track tasks
3. Or collecting using toPandas 

**Workaround:**
1. Reduce the number of partitions if dealing with too many small file/partitions
2. Increase spark.driver.maxResultSize or set it to 0 for unlimited
3. If broadcast joins are the culprit, you can reduce spark.sql.autoBroadcastJoinThreshold to limit the size of broadcast join data

**From stack overflow:**

It seems like the problem is the amount of data you are trying to pull back to to your driver is too large. Most likely you are using the collect method to retrieve all values from a DataFrame/RDD. The driver is a single process and by collecting a DataFrame you are pulling all of that data you had distributed across the cluster back to one node. This defeats the purpose of distributing it! It only makes sense to do this after you have reduced the data down to a manageable amount.

You have two options:

If you really need to work with all that data, then you should keep it out on the executors. Use HDFS and Parquet to save the data in a distributed manner and use Spark methods to work with the data on the cluster instead of trying to collect it all back to one place.

If you really need to get the data back to the driver, you should examine whether you really need ALL of the data or not. If you only need summary statistics then compute that out on the executors before calling collect. Or if you only need the top 100 results, then only collect the top 100.

Update:

There is another reason you can run into this error that is less obvious. Spark will try to send data back the driver beyond just when you explicitly call collect. It will also send back accumulator results for each task if you are using accumulators, data for broadcast joins, and some small status data about each task. If you have LOTS of partitions (20k+ in my experience) you can sometimes see this error. This is a known issue with some improvements made, and more in the works.

The options for getting past if if this is your issue are:

Increase spark.driver.maxResultSize or set it to 0 for unlimited
If broadcast joins are the culprit, you can reduce spark.sql.autoBroadcastJoinThreshold to limit the size of broadcast join data
Reduce the number of partitions

### Scenario 2:
Python UDF errors  
`df = ss.read.parquet(...)
df2 = df.withColumn("A", my_python_udf(sf.col("position.latitude"))
print(df2.groupBy(sf.spark_partition_id()).count().agg(sf.min("count"), sf.max("count"), sf.avg("count")).toPandas())`

[Link: stack overflow](https://stackoverflow.com/questions/62951306/last-task-stuck-forever-only-sometimes?rq=1)

Possible reasons:
1. mismatch between the Python environment used to create the UDF and the Python environment used to execute the UDF.
2. Python process may have encountered some network connectivity issues, or it may have crashed due to a segmentation fault or other low-level error.

Remidiation:
1. Executor has not installed correct versions of python libraries:
   1. One possible cause of this error is that the Python version or the required Python packages used to create the UDF are not installed in the worker nodes of the PySpark cluster. You can try installing the required packages on all the worker nodes using the --py-files option when submitting your PySpark job.
   2. `spark-submit --py-files my_package.tar.gz my_script.py`
2. Do not use the same executor again and serialization issues: 
   1. Alternatively, you can try using a different serialization method for your UDF result by setting the `spark.python.worker.reuse` configuration parameter to False:
   2. This will disable worker reuse and use a new worker for each task, which can help avoid serialization issues. 
   3. `spark.conf.set("spark.python.worker.reuse", "False")`

### Scenario 3:

Your Databricks job reports a failed status, but all Spark jobs and tasks have successfully completed.

**Cause**
1. You have explicitly called spark.stop() or System.exit(0) in your code.
2. If either of these are called, the Spark context is stopped, but the graceful shutdown and handshake with the Databricks job service does not happen.

**Solution**
1. Do not call spark.stop() or System.exit(0) in Spark code that is running on a Databricks cluster.

### Scenario 4:
**Description**: When the Spark driver runs out of memory, exceptions similar to the following exception occur.
'Exception in thread "broadcast-exchange-0" java.lang.OutOfMemoryError: Not enough memory to build and broadcast the table
to all worker nodes. As a workaround, you can either disable broadcast by setting spark.sql.autoBroadcastJoinThreshold to -1
or increase the spark driver memory by setting spark.driver.memory to a higher value'

Solution:
1. crease driver memory: You can increase the amount of memory allocated to the driver by setting `spark.driver.memory` to a higher value.
2. Disable broadcast: You can disable broadcasting by setting `spark.sql.autoBroadcastJoinThreshold` to a very high value or -1. This will prevent Spark from broadcasting tables and will use a shuffle-based join instead.


Note that if your job needs to save results to disk and it fails due to an OutOfMemoryError, it may have written partial results to disk before the failure occurred. You will need to inspect the job logs and output files to determine if this is the case. If partial results have been written, you may be able to restart the job from where it left off and avoid recomputing the partial results.


**how to restart if partial results were written?**  
If partial results were written and the job was terminated due to an error, you can restart the job from the point of failure using checkpointing.

Checkpointing is a mechanism in Spark that allows you to persist RDDs to a reliable storage system so that they can be recovered in case of a failure. Checkpointing is particularly useful when your application has a long lineage, and you want to reduce the cost of recomputing it in case of a failure.

To use checkpointing, you need to enable it in your Spark application using the sparkContext.setCheckpointDir() method, which sets the directory where checkpoint files will be stored. You can then call the RDD.checkpoint() or DataFrame.checkpoint() method to persist RDDs or DataFrames to the checkpoint directory.

Once checkpointing is enabled and the RDDs or DataFrames are checkpointed, you can recover from a failure by restarting the application and setting the checkpoint directory using sparkContext.setCheckpointDir(). When you load the checkpointed RDDs or DataFrames, Spark will use the data stored in the checkpoint files instead of recomputing them.

Note that checkpointing incurs additional I/O overhead, so it should be used judiciously. You should also make sure that you have enough disk space to store the checkpoint files.

### Scenario 5:
"Container killed by YARN for exceeding memory limits. 10.4 GB of 10.4 GB physical memory used" on an EMR cluster with 75GB of memory

[Link: stack overflow](https://stackoverflow.com/questions/40781354/container-killed-by-yarn-for-exceeding-memory-limits-10-4-gb-of-10-4-gb-physic)

[more links: ](https://stackoverflow.com/questions/50587413/container-killed-by-yarn-for-exceeding-memory-limits?noredirect=1&lq=1)
[wiki: ](https://cwiki.apache.org/confluence/display/SPARK/PySpark+Internals)
[Link for 3rd: reason](https://stackoverflow.com/questions/58342902/erratic-occurence-of-container-killed-by-yarn-for-exceeding-memory-limits?rq=1)
[link for 4th reason: ](https://stackoverflow.com/questions/66651606/container-killed-by-yarn-for-exceeding-memory-limits-in-spark-scala?rq=1)

**Reason:** 
1. YARN kills the processes which are taking more memory than they requested which is sum of executorMemoryOverhead and executorMemory.
2. The executor was using 10.4 GB out of 10.4 GB physical memory and YARN decided to kill it to avoid memory contention with other processes on the machine.
3. Using `.coalesce(10)` i.e.uneven partition size: to be the culprit, due to 100 -> 10 reduction in number of partitions without rebalancing the data across them. Doing `.repartition(10)` instead might fix it, at the expense of an extra shuffle. 
4. `--conf spark.dynamicAllocation.enabled=False`  --> As 4th link suggests, problem was not associated with the size of a (too large or small csv) file but static allocation of resources. So while reading small csv --> a large size few partitions were created

**Resolution:**
1. Repartition will help
2. Find out data skew
3. boost the spark.yarn.executor.memoryOverhead parameter, which controls the amount of off-heap memory

## Best reading:
[Link: Amazon EMR tuning guide](https://aws.amazon.com/blogs/big-data/best-practices-for-successfully-managing-memory-for-apache-spark-applications-on-amazon-emr/)

### Scenario 6:
Job failure because the Application Master

Analysis:
1. This happens when you have launched your program in client mode and not in cluster mode
2. Othe reasons are very much simiar to above i.e. why driver goes down?

Workaround:
1. try increasing: `spark.yarn.am.memory`
2. Find out broadcast issues, multiple partitions, accumulators, so many tasks to review etc


### Scenario 7:
Serialized task is too large (Sending data from driver to executors)  
`Serialized task XXX:XXX was XXX bytes, which exceeds max allowed: spark.rpc.message.maxSize (XXX bytes).
Consider increasing spark.rpc.message.maxSize or using broadcast variables for large values.`

Reason:
1. You send some large objects from the driver to executors
2. call parallelize with a large list

Resolution:
1. leverage **broadcast** variables instead
2. Try increasing partitions while parallizing obeject 
   1. `largeList = [...] # This is a large list`  
      `partitionNum = 100 # Increase this number if necessary`  
      `rdd = sc.parallelize(largeList, partitionNum)`  
      `ds = rdd.toDS()`
3. Serilization: Reduce data size while sending from driver to executors. Use **Kryo** instead of the default Java serialization.
4. worst case: set property: `spark.rpc.message.maxSize`